# Object Production and Destruction

> **上级页面**: ["Objects"]("/model/object/")

---






## Introduction

This section covers the beginning and ending of objects' existence, along with the attribution of the artist to the production of the object.

## Base Production Activity

The first activity in an object's lifecycle is its creation, which is described using the `Production` class.  The relationship from the object to the activity is `produced_by`, and the `Production` activity itself follows the general [base activity model](/model/base/#events-and-activities) with the description of time, location and agents.

__Example:__

Similar to the Production example in the basic patterns, "The Night Watch" was created by Rembrandt in Amsterdam in 1642.



In [1]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (The Night Watch painting)
# Who: Rembrandt (the artist)
# What: HumanMadeObject representing "The Night Watch" painting
# Why: To document the artwork and its production information
night_watch = model.HumanMadeObject(ident="nightwatch/5", label="Night Watch by Rembrandt")

# Step 4: Create related objects and relationships
# Production event - when and how the painting was created
production = model.Production()
night_watch.produced_by = production

# Who carried out the production: Rembrandt (the artist)
production.carried_out_by = model.Person(ident="rembrandt", label="Rembrandt")

# When: Time period of creation (1642)
timespan = model.TimeSpan(label="1642")
timespan.begin_of_the_begin = "1642-01-01T00:00:00Z"
timespan.end_of_the_end = "1642-12-31T23:59:59Z"
production.timespan = timespan

# Where: Location of creation (Amsterdam)
production.took_place_at = model.Place(ident="amsterdam", label="Amsterdam")

# Step 5: Display the generated JSON-LD
print(model.factory.toString(night_watch, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://lod.example.org/museum/HumanMadeObject/nightwatch/5",
  "type": "HumanMadeObject",
  "_label": "Night Watch by Rembrandt",
  "produced_by": {
    "type": "Production",
    "timespan": {
      "type": "TimeSpan",
      "_label": "1642",
      "begin_of_the_begin": "1642-01-01T00:00:00Z",
      "end_of_the_end": "1642-12-31T23:59:59Z"
    },
    "took_place_at": [
      {
        "id": "http://lod.example.org/museum/Place/amsterdam",
        "type": "Place",
        "_label": "Amsterdam"
      }
    ],
    "carried_out_by": [
      {
        "id": "http://lod.example.org/museum/Person/rembrandt",
        "type": "Person",
        "_label": "Rembrandt"
      }
    ]
  }
}




## Techniques and Classifications

We distinguish between techniques used to create the artwork, and other classifications. Techniques have their own property (`technique` rather than `classified_as`) and can thus easily be distinguished from other classifications.

### Techniques

If there is a particular technique known to have been used in the creation of the object, this can be expressed using the `technique` property, referring to a controlled vocabulary term for the technique. This should be used to capture specific techniques or methods, and the base `classified_as` property used for more general classifications of the activity.  More general classifications are more common on individual roles, discussed below.
 
__Example:__

The [sculpture "Bust of a Man"](https://collections.britishart.yale.edu/catalog/tms:54430) was created by Francis Hardwood using the sculpting technique.



In [2]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Bust of a Man sculpture)
# Who: Francis Harwood (the sculptor)
# What: Sculpture object representing "Bust of a Man"
# Why: To document the sculpture and its production technique
sculpture = vocab.Sculpture(ident="bust/1", label="Bust of a Man")

# Step 4: Create related objects and relationships
# Production event
production = model.Production()
sculpture.produced_by = production

# Who carried out the production: Francis Harwood (the sculptor)
production.carried_out_by = model.Person(
    ident="http://vocab.getty.edu/ulan/500015886",
    label="Francis Harwood"
)

# What technique was used: Sculpting
production.technique = vocab.instances['sculpting']

# Step 5: Display the generated JSON-LD
print(model.factory.toString(production, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "type": "Production",
  "technique": [
    {
      "id": "http://vocab.getty.edu/aat/300264383",
      "type": "Type",
      "_label": "Sculpting"
    }
  ],
  "carried_out_by": [
    {
      "id": "http://vocab.getty.edu/ulan/500015886",
      "type": "Person",
      "_label": "Francis Harwood"
    }
  ]
}




## Multiple Artists (with Different Roles)

If there are multiple artists collaborating on the same piece of artwork, then we follow the partitioning pattern of creating separate parts of the main `Production` activity. Each of these components captures the details of one particular artist's role in the production of the object. This allows us to assert different properties for each artist's contribution, including different times, techniques, roles, locations or influences.

For consistency, it is RECOMMENDED that this pattern also be used for production activities when only one artist is known, such that it is easier to add further contributors to the work without restructuring the content. However this is not required, to ensure compatibility with existing systems.

Note that these parts of productions can also have statements associated with them, describing the role in more detail beyond just the classification, technique or other structured data.

__Example:__

A [painted textile called "RÜN"](https://artgallery.yale.edu/collections/objects/153900), where the linen was hand-woven by Sarah Parke and then painted by Mark Barrow.




In [3]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (RUN painted textile)
# Who: Sarah Parke (weaver) and Mark Barrow (painter)
# What: Painting object representing "RUN" collaborative artwork
# Why: To document multiple artists with different roles in production
artwork = vocab.Painting(ident="run/1", label="RUN")

# Step 4: Create related objects and relationships
# Main production event
production = model.Production()
artwork.produced_by = production

# First artist contribution: Mark Barrow (painting)
painting_activity = model.Production()
painting_activity.technique = vocab.instances['painting']
painting_activity.carried_out_by = model.Person(ident="barrow", label="Mark Barrow")
production.part = painting_activity

# Second artist contribution: Sarah Parke (hand weaving)
weaving_activity = model.Production()
weaving_activity.technique = model.Type(
    ident="http://vocab.getty.edu/aat/300053643",
    label="hand weaving"
)
weaving_activity.carried_out_by = model.Person(ident="parke", label="Sarah Parke")
production.part = weaving_activity

# Step 5: Display the generated JSON-LD
print(model.factory.toString(production, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "type": "Production",
  "part": [
    {
      "type": "Production",
      "technique": [
        {
          "id": "http://vocab.getty.edu/aat/300054216",
          "type": "Type",
          "_label": "Painting"
        }
      ],
      "carried_out_by": [
        {
          "id": "http://lod.example.org/museum/Person/barrow",
          "type": "Person",
          "_label": "Mark Barrow"
        }
      ]
    },
    {
      "type": "Production",
      "technique": [
        {
          "id": "http://vocab.getty.edu/aat/300053643",
          "type": "Type",
          "_label": "hand weaving"
        }
      ],
      "carried_out_by": [
        {
          "id": "http://lod.example.org/museum/Person/parke",
          "type": "Person",
          "_label": "Sarah Parke"
        }
      ]
    }
  ]
}




## Objects Related to the Production

Other objects can play critical roles in the production of artwork, such as copying or being inspired by another artwork, or the use of the same source to create an artwork, either mechanically or manually, such as the negative used for printing a photograph or the plate used to print an etching.


### Inspirations, Studies or Copies

Some artworks are copies of, or clearly directly inspired by, others.  This relationship with another work can be captured with the `influenced_by` property of the `Production` activity. The copy could be from memory or with the copied object physically present, and it could be a faithful reproduction or merely recognizably similar. This includes studies done for the final version of the work. The exact nature of the influence is not captured in the model.

__Example:__

In 1964, Deane Keller created [a copy](https://artgallery.yale.edu/collections/objects/54339) of Daniel Huntington's [portrait of James Dwight Dana](https://artgallery.yale.edu/collections/objects/52687), from 1858. 



In [4]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Copy of Huntington Portrait)
# Who: Deane Keller (the copier)
# What: Painting representing a copy of Daniel Huntington's portrait
# Why: To document the relationship between the copy and the original artwork
copy_portrait = vocab.Painting(
    ident="kellerdana/1",
    label="Copy of Huntington Portrait"
)

# Step 4: Create related objects and relationships
# Production event
production = model.Production()
copy_portrait.produced_by = production

# What influenced the production: Original Huntington Portrait
original_portrait = model.HumanMadeObject(
    ident="huntingtondana",
    label="Huntington Portrait of Dana"
)
production.influenced_by = original_portrait

# Who carried out the production: Deane Keller (the copier)
production.carried_out_by = model.Person(ident="keller", label="Deane Keller")

# Step 5: Display the generated JSON-LD
print(model.factory.toString(production, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "type": "Production",
  "carried_out_by": [
    {
      "id": "http://lod.example.org/museum/Person/keller",
      "type": "Person",
      "_label": "Deane Keller"
    }
  ],
  "influenced_by": [
    {
      "id": "http://lod.example.org/museum/HumanMadeObject/huntingtondana",
      "type": "HumanMadeObject",
      "_label": "Huntington Portrait of Dana"
    }
  ]
}




### Reproduction from an Identifiable Source

Many objects are created directly from a source, such as a photograph being printed from a negative, a print created from a woodcut, or a sculpture made from a cast.  The use of the particular source object can be captured as part of the description of the `Production` of the object using the `used_specific_object` property.

Note that all of the art objects created from the same source will show the same image, be it flat or three dimensional. The source also shows the same image, albeit likely somehow reversed. The image is modeled as a `VisualItem` that all of the physical objects show, allowing us to group the objects together. For more information about the work, see the section on [aboutness](../aboutness/).

Equipment or tools such as a particular camera or palette would also be modeled with the same property `used_specific_object`, however would not (of course) show the same visual item as the main work.

__Example:__

Copies of a photograph, taken by Alfred Stieglitz of Georgia O'Keeffe, are printed from the same negative at different times and now owned by different organizations: [Yale University Art Gallery](https://artgallery.yale.edu/collections/objects/198690), [National Gallery of Art](https://www.nga.gov/collection/art-object-page.60057.html), and the [Georgia O'Keeffe Museum](https://collections.okeeffemuseum.org/object/6627/)



In [5]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (GOK 1918 photograph at Georgia O'Keeffe Museum)
# Who: Alfred Stieglitz (photographer), Georgia O'Keeffe Museum (owner)
# What: Photograph object representing a print from the same negative
# Why: To document prints created from the same source negative
photo_gokm = vocab.Photograph(
    ident="okeeffe-gok/1",
    label="GOK 1918, GOKM"
)
photo_gokm.identified_by = vocab.AccessionNumber(content="2014.3.78")

# Step 4: Create related objects and relationships
# Production event (printing from negative)
production = model.Production(label="Printing of Photograph")
photo_gokm.produced_by = production

# What specific object was used: The negative source
negative = model.HumanMadeObject(
    ident="okeeffe-negative",
    label="Negative of GOK 1918"
)
production.used_specific_object = negative

# What visual content is shown: The shared visual item
visual_content = model.VisualItem(
    ident="okeeffe",
    label="Visual Content of GOK 1918"
)
photo_gokm.shows = visual_content

# Step 5: Display the generated JSON-LD
print(model.factory.toString(production, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "type": "Production",
  "_label": "Printing of Photograph",
  "used_specific_object": [
    {
      "id": "http://lod.example.org/museum/HumanMadeObject/okeeffe-negative",
      "type": "HumanMadeObject",
      "_label": "Negative of GOK 1918"
    }
  ]
}


In [6]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (GOK 1918 photograph at Yale University Art Gallery)
# Who: Alfred Stieglitz (photographer), Yale University Art Gallery (owner)
# What: Another photograph print from the same negative
# Why: To demonstrate how multiple prints from the same negative are documented
photo_yuag = vocab.Photograph(
    ident="okeeffe-yuag/1",
    label="GOK 1918, YUAG"
)
photo_yuag.identified_by = vocab.AccessionNumber(content="2016.101.242")

# Step 4: Create related objects and relationships
# Production event (printing from negative)
production = model.Production(label="Printing of Photograph")
photo_yuag.produced_by = production

# What specific object was used: The same negative source
negative = model.HumanMadeObject(
    ident="okeeffe-negative",
    label="Negative of GOK 1918"
)
production.used_specific_object = negative

# What visual content is shown: The shared visual item
visual_content = model.VisualItem(
    ident="okeeffe",
    label="Visual Content of GOK 1918"
)
photo_yuag.shows = visual_content

# Step 5: Display the generated JSON-LD
print(model.factory.toString(production, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "type": "Production",
  "_label": "Printing of Photograph",
  "used_specific_object": [
    {
      "id": "http://lod.example.org/museum/HumanMadeObject/okeeffe-negative",
      "type": "HumanMadeObject",
      "_label": "Negative of GOK 1918"
    }
  ]
}




__Example:__

A [print](https://collections.britishart.yale.edu/catalog/tms:6141) at the Yale Center for British Art, Chaucer's Canterbury Pilgrims, made from a specific [copper plate](https://artgallery.yale.edu/collections/objects/11787) held at the Yale University Art Gallery.



In [7]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Chaucer's Canterbury Pilgrims print)
# Who: Unknown printmaker, Yale Center for British Art (owner)
# What: Print object made from a specific copper plate
# Why: To document a print created from an identifiable source object
print_artwork = vocab.Print(
    ident="ccp/1",
    label="Chaucer's Canterbury Pilgrims"
)

# Step 4: Create related objects and relationships
# Production event (printing from plate)
production = model.Production(label="Printing from Plate")
print_artwork.produced_by = production

# What specific object was used: The copper plate source
production.used_specific_object = model.HumanMadeObject(
    ident="ccp-plate",
    label="Plate for CCP"
)

# Step 5: Display the generated JSON-LD
print(model.factory.toString(production, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "type": "Production",
  "_label": "Printing from Plate",
  "used_specific_object": [
    {
      "id": "http://lod.example.org/museum/HumanMadeObject/ccp-plate",
      "type": "HumanMadeObject",
      "_label": "Plate for CCP"
    }
  ]
}


In [8]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (The copper plate itself)
# Who: Unknown plate maker, Yale University Art Gallery (owner)
# What: HumanMadeObject representing the copper plate source
# Why: To document the physical object (plate) used to create prints
copper_plate = model.HumanMadeObject(
    ident="ccp-plate/1",
    label="Plate for CCP"
)

# Step 4: Create related objects and relationships
# What material the plate is made of: Copper
copper_plate.made_of = vocab.instances['copper']

# Step 5: Display the generated JSON-LD
print(model.factory.toString(copper_plate, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://lod.example.org/museum/HumanMadeObject/ccp-plate/1",
  "type": "HumanMadeObject",
  "_label": "Plate for CCP",
  "made_of": [
    {
      "id": "http://vocab.getty.edu/aat/300011020",
      "type": "Material",
      "_label": "copper"
    }
  ]
}




## Cause of Production

Artworks are frequently commissioned, where an artist agrees to create the artwork and the commissioner agrees to compensate the artist for his efforts. The modeling of the [commission](/model/provenance/promises/#commissions-for-artwork) is described in the Provenance section, as an exchange of promises, and potentially also Payments. The object can refer to this Activity in its Production with the `caused_by` property.

__Example:__

The production of the [Nuveen Painting](https://artgallery.yale.edu/collections/objects/290048) was caused by its commissioning.



In [9]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Nuveen Painting)
# Who: Jim Dine (artist), unknown commissioner
# What: HumanMadeObject representing "The Nuveen Painting"
# Why: To document a painting caused by a commission agreement
nuveen_painting = model.HumanMadeObject(
    ident="nuveen/1",
    label="Nuveen Painting"
)
nuveen_painting.identified_by = vocab.PrimaryName(content="The Nuveen Painting")

# Step 4: Create related objects and relationships
# Production event
production = model.Production()
nuveen_painting.produced_by = production

# Who carried out the production: Jim Dine (the artist)
production.carried_out_by = model.Person(ident="dine", label="Jim Dine")

# What caused the production: The commission activity
commission_activity = model.Activity(
    ident="nuveen_commission",
    label="Commission"
)
production.caused_by = commission_activity

# Step 5: Display the generated JSON-LD
print(model.factory.toString(nuveen_painting, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://lod.example.org/museum/HumanMadeObject/nuveen/1",
  "type": "HumanMadeObject",
  "_label": "Nuveen Painting",
  "identified_by": [
    {
      "type": "Name",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300404670",
          "type": "Type",
          "_label": "Primary Name"
        }
      ],
      "content": "The Nuveen Painting"
    }
  ],
  "produced_by": {
    "type": "Production",
    "carried_out_by": [
      {
        "id": "http://lod.example.org/museum/Person/dine",
        "type": "Person",
        "_label": "Jim Dine"
      }
    ],
    "caused_by": [
      {
        "id": "http://lod.example.org/museum/Activity/nuveen_commission",
        "type": "Activity",
        "_label": "Commission"
      }
    ]
  }
}




## Unidentified or Unknown Artist

Many objects are created by an unknown or unidentified artist. While it is possible to have a different Person record with "Unidentified Artist" as the name, this quickly creates a huge number of identified people in the system with only a single reference to each. Instead, it is recommended to create a Group record to represent all unidentified artists, and then each object would have its Production `carried_out_by` the Group, meaning one (or more) people within that artificial set of people.

If some features of the artist is know, for example their nationality or the century in which they were active, these can be additional Groups with these properties.

__Example:__

The [Coppa Amatoria](https://artgallery.yale.edu/collections/objects/138452) was created by an unidentified artist or artists from Italy.



In [10]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Coppa Amatoria)
# Who: Unidentified Italian artist(s)
# What: HumanMadeObject representing "Coppa Amatoria"
# Why: To document an artwork created by unidentified Italian artists
coppa = model.HumanMadeObject(
    ident="coppa/1",
    label="Coppa Amatoria"
)
coppa.identified_by = vocab.PrimaryName(content="Coppa Amatoria")

# Step 4: Create related objects and relationships
# Production event
production = model.Production()
coppa.produced_by = production

# Who carried out the production: Unidentified Italian artists (represented as a Group)
production.carried_out_by = model.Group(
    ident="unknown_italian",
    label="Unidentified Italian"
)

# Step 5: Display the generated JSON-LD
print(model.factory.toString(coppa, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://lod.example.org/museum/HumanMadeObject/coppa/1",
  "type": "HumanMadeObject",
  "_label": "Coppa Amatoria",
  "identified_by": [
    {
      "type": "Name",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300404670",
          "type": "Type",
          "_label": "Primary Name"
        }
      ],
      "content": "Coppa Amatoria"
    }
  ],
  "produced_by": {
    "type": "Production",
    "carried_out_by": [
      {
        "id": "http://lod.example.org/museum/Group/unknown_italian",
        "type": "Group",
        "_label": "Unidentified Italian"
      }
    ]
  }
}





## Attribution Qualifiers

### Influenced By an Artist

If there is some connection between the production of the object and someone who was not the artist directly but influenced the production, then the `influenced_by` property can be used to reference that person.  These are often expressed as "after", "in the style of", or "in the manner of" attributions -- they qualify the attribution by relating the production to someone (likely as embodied by their work, rather than through a personal connection) that directly influenced it.

__Example:__

The painting "Wash Day" was intentionally created in the manner of Winslow Homer.



In [11]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Wash Day painting)
# Who: Unknown artist, influenced by Winslow Homer
# What: Painting object representing "Wash Day"
# Why: To document a painting created in the manner of another artist
wash_day = vocab.Painting(
    ident="washday/1",
    label="Wash Day"
)
wash_day.identified_by = vocab.PrimaryName(content="Wash Day")

# Step 4: Create related objects and relationships
# Production event
production = model.Production()
wash_day.produced_by = production

# Who influenced the production: Winslow Homer (the influential artist)
production.influenced_by = model.Person(
    ident="whomer",
    label="Winslow Homer"
)

# Step 5: Display the generated JSON-LD
print(model.factory.toString(production, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "type": "Production",
  "influenced_by": [
    {
      "id": "http://lod.example.org/museum/Person/whomer",
      "type": "Person",
      "_label": "Winslow Homer"
    }
  ]
}




### Attribution of a Group Related to an Artist

Even if the artist's or artists' identity is not known exactly, the person or people may be known to have been part of a group, such as the workshop of a more famous "master". In this case, the `Group` that represents the workshop can be the actor that carries out the `Production`: this does not mean that every member of the group participated, just that at least one of them did, in the same way that saying that a document was written or published by an organization does not imply all employees contributed to that effort.

We can use the `influenced_by` property on the `Formation` (the creation) of the group to connect it to a known person -- the "master" of the workshop in the example use case. The "master" might not have participated in the group, or even been alive when it was formed, and hence does not necessarily form the group or was even a member of it. 

This approach can be used for workshops, studios, the set of pupils, followers, and so forth. It is not that the entire Group created the object, but that one or more of them did. 

__Example:__

The "Bust of a Man" object described above was previously thought to have been created by the Studio of Francis Harwood, a Group. The example below is the record for the Group.
 


In [12]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Studio of Francis Harwood)
# Who: Francis Harwood (influential master), unknown studio members
# What: Group representing the studio of Francis Harwood
# Why: To document a workshop/studio group influenced by a master artist
studio_group = vocab.Studio(
    ident="harwoodstudio/1",
    label="Studio of Francis Harwood"
)

# Step 4: Create related objects and relationships
# Formation event of the studio group
formation = model.Formation()
studio_group.formed_by = formation

# Who influenced the formation: Francis Harwood (the master artist)
formation.influenced_by = model.Person(
    ident="http://vocab.getty.edu/ulan/500015886",
    label="Francis Harwood"
)

# Step 5: Display the generated JSON-LD
print(model.factory.toString(formation, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "type": "Formation",
  "influenced_by": [
    {
      "id": "http://vocab.getty.edu/ulan/500015886",
      "type": "Person",
      "_label": "Francis Harwood"
    }
  ]
}




### Uncertain or Changing Attributions

A piece of information associated with historical artworks that can change as research and understanding improves is the identity of the artist that produced it. The actor that is referenced in `carried_out_by` is the current opinion, but previous attributions can still be recorded. The pattern used for this is described in more detail in the [assertions](/model/assertion/) section of the documentation.


## Production by Removal

It is possible for an object to come into documentary existence when it is removed from a larger object.  The production of the part is simply part of the production of the whole, but until it is removed, it does not need a separate identity or existence.

This occurs reasonably frequently, for both valid and unscrupulous motivations.  In the work of conservation, it is often necessary to remove a tiny flake of an object to experiment with, before applying the method to the whole.  If there are unexpected side effects of the experiment, then the whole object is saved at the expense of an unnoticeable change. It is important to know that the sample was produced in the past and removed in the present, rather than it was created in the present.

A second scenario when this occurs is unfortunately common.  If an object, such as a medieval manuscript, can be sold for a higher profit by splitting it up into parts and selling each part individually, then unscrupulous sellers will do just that. Rather than sell an innocuous book of hours to a single buyer, instead each illumination can be sold individually and then the remaining text-bearing pages either dumped or sold over time at a greatly reduced price.  This dispersal of manuscripts still occurs today.

In order to model this, instead of a `Production`, the object is `removed_by` a `PartRemoval` activity.  That activity is just like all other activities, however it also has a `diminished` property that refers to the whole object from which the part was removed.  It is not normally useful to have a separate `Production` for the part, if the information about the source object it was removed from is known.


__Example:__

The collection item is a page from a gradual (manuscript) by the Master of the Libro dei Notai. [Source](https://artgallery.yale.edu/collections/objects/51500)



In [13]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Page from a Gradual)
# Who: Master of the Libro dei Notai (creator), unknown agent who removed the page
# What: Page object removed from a larger manuscript
# Why: To document an object that came into existence by being removed from a whole
page = vocab.Page(
    ident="gradualpage/1",
    label="Page from a Gradual"
)

# Step 4: Create related objects and relationships
# PartRemoval event (when the page was removed from the manuscript)
removal = model.PartRemoval()

# What was diminished: The original gradual manuscript
whole_manuscript = model.HumanMadeObject(
    ident="gradual",
    label="Gradual"
)
removal.diminished = whole_manuscript

page.removed_by = removal

# Step 5: Display the generated JSON-LD
print(model.factory.toString(removal, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "type": "PartRemoval",
  "diminished": {
    "id": "http://lod.example.org/museum/HumanMadeObject/gradual",
    "type": "HumanMadeObject",
    "_label": "Gradual"
  }
}




## Discovery versus Production

If the object enters into documented history by being found, rather than created, then instead of a `Production` entry, there is an `Encounter`. This would be appropriate for fossils, gemstones, or other natural history items, along with objects that were created but no information is available about that production or the "discovery" of the object is the first significant entry in the item's provenance. This pattern may only be used for the discovery of the object, rather than other sorts of encounters.

Encounters may also be recorded as separate events, as described in the [provenance section](../../provenance/encounters/). The separate event must be used for any encounter that is not a discovery.

Instead of `produced_by`, the property used is `encountered_by`, and the class of the activity is `Encounter`. All of the other patterns are the same, including where the encounter took place, who encountered it, when it occurred and so forth. The role pattern also applies, with the `part`s of the encounter also being `Encounter`s.

__Example:__

The fossil Torosaurus (VP.04072) was encountered in 1891 by John Bell Hatcher.



In [14]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Torosaurus fossil)
# Who: John Bell Hatcher (discoverer)
# What: HumanMadeObject representing the discovered fossil
# Why: To document a natural history object that entered recorded history via discovery
torosaurus = model.HumanMadeObject(
    ident="torosaurus/1",
    label="Torosaurus Gladius"
)

# Step 4: Create related objects and relationships
# Encounter event (discovery of the fossil)
encounter = model.Encounter()

# Who carried out the encounter: John Bell Hatcher (the discoverer)
encounter.carried_out_by = model.Person(
    ident="hatcher",
    label="John Bell Hatcher"
)

# When: The discovery occurred in 1891
timespan = model.TimeSpan()
timespan.begin_of_the_begin = "1891-01-01T00:00:00Z"
timespan.end_of_the_end = "1891-12-31T23:59:59Z"
encounter.timespan = timespan

torosaurus.encountered_by = encounter

# Step 5: Display the generated JSON-LD
print(model.factory.toString(torosaurus, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://lod.example.org/museum/HumanMadeObject/torosaurus/1",
  "type": "HumanMadeObject",
  "_label": "Torosaurus Gladius",
  "encountered_by": [
    {
      "type": "Encounter",
      "timespan": {
        "type": "TimeSpan",
        "begin_of_the_begin": "1891-01-01T00:00:00Z",
        "end_of_the_end": "1891-12-31T23:59:59Z"
      },
      "carried_out_by": [
        {
          "id": "http://lod.example.org/museum/Person/hatcher",
          "type": "Person",
          "_label": "John Bell Hatcher"
        }
      ]
    }
  ]
}




## Destruction

The end of the provenance chain of an object is when it is known to have been destroyed.  Loss of the object leaves the chain open ended as it might be recovered in the future, however if the object is destroyed then there is no coming back. Objects should, thus, only be recorded as destroyed if they are known to be so.

The model uses a `Destruction` class that represents the going out of existence of the object. A destruction only applies to a single object, and thus every object has their own destruction, even if that destruction was caused by the same wider event or activity.  This means that the Destruction is not `carried_out_by` an actor, but instead (as in the following section) another activity causes the destruction.

__Example:__

The painting "Le Peintre" by Picasso was destroyed (in a plane crash) on September 2nd, 1998 at about 10:30pm. [Source](https://en.wikipedia.org/wiki/Swissair_Flight_111)



In [15]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Le Peintre by Picasso)
# Who: Pablo Picasso (artist), Swissair Flight 111 crash (destruction cause)
# What: Painting object that was destroyed
# Why: To document the end of an object's provenance chain through destruction
painting = vocab.Painting(
    ident="lepeintre/1",
    label="Le Peintre by Picasso"
)

# Step 4: Create related objects and relationships
# Destruction event
destruction = model.Destruction(label="Destruction of Le Peintre")
painting.destroyed_by = destruction

# When: The destruction occurred (September 2nd, 1998, approximately 10:30pm)
timespan = model.TimeSpan()
timespan.begin_of_the_begin = "1998-09-02T22:20:00Z"
timespan.end_of_the_end = "1998-09-02T22:40:00Z"
destruction.timespan = timespan

# Step 5: Display the generated JSON-LD
print(model.factory.toString(destruction, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "type": "Destruction",
  "_label": "Destruction of Le Peintre",
  "timespan": {
    "type": "TimeSpan",
    "begin_of_the_begin": "1998-09-02T22:20:00Z",
    "end_of_the_end": "1998-09-02T22:40:00Z"
  }
}




### Cause of Destruction

As discussed above, the same event or activity could be the cause of the destruction of many individual objects. A museum burning down or otherwise being destroyed would destroy many objects, including the building itself. On a smaller scale, an individual destruction could be caused by a particular activity of an individual. 

In order to distinguish between the destruction itself and its cause, we use the `caused_by` relationship between the Destruction and the Activity or Event that caused it. The event should be described and classified as any other event in the model.

__Example:__

Le Peintre was destroyed because of the plane crashing. (Which also would have caused the destruction of the plane, the death of the crew, passengers and many other consequences)



In [16]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Le Peintre by Picasso)
# Who: Pablo Picasso (artist), Swissair Flight 111 crash (destruction cause)
# What: Painting object destroyed by a plane crash
# Why: To document the cause of destruction separately from the destruction event itself
painting = vocab.Painting(
    ident="lepeintre/2",
    label="Le Peintre by Picasso"
)

# Step 4: Create related objects and relationships
# Destruction event
destruction = model.Destruction(label="Destruction of Le Peintre")
painting.destroyed_by = destruction

# When: The destruction occurred
timespan = model.TimeSpan()
timespan.begin_of_the_begin = "1998-09-02T22:20:00Z"
timespan.end_of_the_end = "1998-09-02T22:40:00Z"
destruction.timespan = timespan

# What caused the destruction: The plane crash event
crash_event = model.Event(
    ident="sr111crash",
    label="Crash of Swiss Air 111"
)
destruction.caused_by = crash_event

# Step 5: Display the generated JSON-LD
print(model.factory.toString(destruction, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "type": "Destruction",
  "_label": "Destruction of Le Peintre",
  "timespan": {
    "type": "TimeSpan",
    "begin_of_the_begin": "1998-09-02T22:20:00Z",
    "end_of_the_end": "1998-09-02T22:40:00Z"
  },
  "caused_by": [
    {
      "id": "http://lod.example.org/museum/Event/sr111crash",
      "type": "Event",
      "_label": "Crash of Swiss Air 111"
    }
  ]
}
